In [ ]:
%pip install requests
%pip install pandas

In [ ]:
# Imports de librerías requeridas

import requests
import json
import pandas as pd
import os



In [ ]:
# Url donde obtendremos la información del histórico de precios de bitcoin vs usd 3 años hacia atrás.
url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=1095'

# Pretición get a la url
response = requests.get(url)

# Formatear la información a JSON.
data = response.json()

# Extraer los precios de cierre y las fechas del JSON
prices_info = data['prices']

prices = []
timestamps = []

# Convertir los timestamps de milisegundos a segundos y almacenar los precios en el array de precios
for i in prices_info:
  prices.append(i[1])
  timestamps.append(i[0]/1000)  

# print(prices, timestamps)

# Crear el DataFrame utilizando pandas
df = pd.DataFrame(prices, columns=['Price'])
df['Date'] = pd.to_datetime(timestamps, unit='s')

# # Listar los primeros registros del DF para comprobar que se formateo correctamente.
print(df.head())

# Insertar información a la db

In [ ]:
from sqlalchemy import create_engine

host = os.environ.get("host")
port = os.environ.get("port")
database = os.environ.get("database")
user = os.environ.get("user")
password = os.environ.get("password")

table_name = "bicoin_prices"

# Crear la conexión a la db
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

# Insertar datos desde el dataframe en la db
df.to_sql(
    table_name, 
    engine, 
    if_exists='append', 
    index=False
)